In [10]:
import os
from pytse_client.download import download_financial_indexes
import pandas as pd
import jalali_pandas
import math
from scipy.stats.mstats import winsorize
import numpy as np
import warnings
warnings.filterwarnings("ignore")

def return_calculator(df):
    period_return = 1
    li = list(df)
    for day, _ in df.iteritems():
        idx = df.index.get_loc(day)
        elem = li[idx]
        if (math.isnan(elem)):
            elem = outlier_handler(elem)
            if elem == False:
                continue
        period_return = period_return * (1+elem)
    period_return = period_return - 1
    return period_return

def outlier_handler(ret):
    if (math.isnan(ret)):
        return False


desktop = os.path.join(os.path.join(os.path.expanduser('~')), 'Desktop') 
data_path = os.path.join(desktop, "sector_slc", "dataset")
sectors_data_path = os.path.join(data_path, "sectors")
market_data_path = os.path.join(data_path, "market")
market_other_data_path = os.path.join(data_path, "market_other_data")
list_of_sector_files = os.listdir(sectors_data_path)
list_of_market_files = os.listdir(market_data_path)
list_of_market_other_data_files = os.listdir(market_other_data_path)
market_data_processed_path = os.path.join(data_path, "market_data_processed")
sectors_data_processed_path = os.path.join(data_path, "sectors_data_processed")
list_of_sector_processed_files = os.listdir(sectors_data_processed_path)
list_of_sector_processed_files = list_of_sector_processed_files[1:]
list_of_market_processed_files = os.listdir(market_data_processed_path)
list_of_market_processed_files = list_of_market_processed_files[1:]
dataset_path = os.path.join(data_path, "dataset")

In [2]:
for file_name in list_of_sector_files:
    
    data = pd.read_excel(sectors_data_path+ "\\" + file_name)
    first_row_with_pe_ttm = data[data["P/E-ttm"] != "-"].iloc[-1].name 
    data = data[:first_row_with_pe_ttm + 1]
    first_row_with_trade_value = data[data["ارزش معاملات"] != "-"].iloc[-1].name 
    data = data[:first_row_with_trade_value + 1]
    first_row_with_pct_change = data[data["درصد تغییر"] != "-"].iloc[-1].name 
    data = data[:first_row_with_pct_change + 1]
    data = data[["تاریخ شمسی","درصد تغییر","ارزش معاملات","P/E-ttm"]]
    data = data.replace('-', np.nan)
    data["date"] = data["تاریخ شمسی"].jalali.parse_jalali("%Y-%m-%d")
    data["month"] = data["date"].jalali.month
    data["year"] = data["date"].jalali.year
    data["quarter"] = data["date"].jalali.quarter
    data["weekday"] = data["date"].jalali.weekday
    data["day"] = data["date"].jalali.day
    data = data.drop(["تاریخ شمسی"],axis=1)
    data.rename(columns = {'درصد تغییر':'dayli_return', 'ارزش معاملات':'trade_value'}, inplace = True)
    last_years_month = data["month"][0]
    last_year = data["year"][0]
    first_years_month = data["month"].iloc[-1]
    first_year = data["year"].iloc[-1]

    empty_li_month = []
    empty_li_year = []
    for year in range(first_year, last_year + 1):

        if year == first_year:
            empty_li_month += [i for i in range(first_years_month,13)]
            empty_li_year += [year] * (13-first_years_month)

        elif year == last_year:
            empty_li_month += [i for i in range(1,last_years_month+1)]
            empty_li_year += [year] * (last_years_month)

        else:
            empty_li_month += [i for i in range(1,13)]
            empty_li_year += [year] * 12

    monthly_data = pd.DataFrame(columns=["year","quarter","month","monthly_return","monthly_trade_value","P/E-ttm"])
    monthly_data["year"] = empty_li_year
    monthly_data["month"] = empty_li_month

    for year in range(first_year, last_year + 1):

        if year == first_year:
            for month in range(first_years_month, 13):
                idx = monthly_data[(monthly_data.year == year) & (monthly_data.month == month)].index.item()
                ret = return_calculator(data[(data["month"] == month) & (data["year"] == year)]["dayli_return"])
                monthly_trade_value = data[(data["month"] == month) & (data["year"] == year)]["trade_value"].mean()
                month_pe_ttm = data[(data["month"] == month) & (data["year"] == year)]["P/E-ttm"].dropna().mean()
                monthly_data["quarter"][idx] = data[(data["month"] == month) & (data["year"] == year)].quarter.mean()
                monthly_data["monthly_return"][idx] = ret
                monthly_data["monthly_trade_value"][idx] = monthly_trade_value
                monthly_data["P/E-ttm"][idx] = month_pe_ttm

        elif year == last_year:
            for month in range(1, last_years_month + 1):
                idx = monthly_data[(monthly_data.year == year) & (monthly_data.month == month)].index.item()
                ret = return_calculator(data[(data["month"] == month) & (data["year"] == year)]["dayli_return"])
                monthly_trade_value = data[(data["month"] == month) & (data["year"] == year)]["trade_value"].mean()
                month_pe_ttm = data[(data["month"] == month) & (data["year"] == year)]["P/E-ttm"].dropna().mean()
                monthly_data["quarter"][idx] = data[(data["month"] == month) & (data["year"] == year)].quarter.mean()
                monthly_data["monthly_return"][idx] = ret
                monthly_data["monthly_trade_value"][idx] = monthly_trade_value
                monthly_data["P/E-ttm"][idx] = month_pe_ttm

        else:
            for month in range(1, 13):
                idx = monthly_data[(monthly_data.year == year) & (monthly_data.month == month)].index.item()
                ret = return_calculator(data[(data["month"] == month) & (data["year"] == year)]["dayli_return"])
                monthly_trade_value = data[(data["month"] == month) & (data["year"] == year)]["trade_value"].mean()
                month_pe_ttm = data[(data["month"] == month) & (data["year"] == year)]["P/E-ttm"].dropna().mean()
                monthly_data["quarter"][idx] = data[(data["month"] == month) & (data["year"] == year)].quarter.mean()
                monthly_data["monthly_return"][idx] = ret
                monthly_data["monthly_trade_value"][idx] = monthly_trade_value
                monthly_data["P/E-ttm"][idx] = month_pe_ttm

    monthly_data.to_excel(sectors_data_processed_path + "\\" + file_name)

market_df = pd.DataFrame()
for file_name in list_of_market_files:
    data = pd.read_excel(market_data_path+ "\\" + file_name)
    file_name = file_name.split(".")[0]
    data.rename(columns = {'مقدار':file_name, "تاریخ شمسی":"date"}, inplace = True)
    data = data.drop(["تاریخ میلادی"],axis=1)
    data["date"] = data["date"].jalali.parse_jalali("%Y-%m-%d")
    data["month"] = data["date"].jalali.month
    data["year"] = data["date"].jalali.year
    data["quarter"] = data["date"].jalali.quarter
    data["weekday"] = data["date"].jalali.weekday
    data["day"] = data["date"].jalali.day
    if len(market_df) == 0:
        market_df = data
    else:
        market_df = market_df.merge(data,on="date",how="inner")
    
market_df = market_df[["date","market_pe","overall_index","trade_value","usd","nima_usd"]]
market_df["month"] = market_df["date"].jalali.month
market_df["year"] = market_df["date"].jalali.year
market_df["quarter"] = market_df["date"].jalali.quarter
market_df["weekday"] = market_df["date"].jalali.weekday
market_df["day"] = market_df["date"].jalali.day
market_df["diff_nima_usd"] = market_df["nima_usd"] / market_df["usd"] - 1
market_df["usd"] = market_df["usd"].pct_change() 
market_df["overall_index"] = market_df["overall_index"].pct_change()

first_years_month = market_df["month"][0]
first_year = market_df["year"][0]
last_years_month = market_df["month"].iloc[-1]
last_year = market_df["year"].iloc[-1]

empty_li_month = []
empty_li_year = []
for year in range(first_year, last_year + 1):

    if year == first_year:
        empty_li_month += [i for i in range(first_years_month,13)]
        empty_li_year += [year] * (13-first_years_month)

    elif year == last_year:
        empty_li_month += [i for i in range(1,last_years_month+1)]
        empty_li_year += [year] * (last_years_month)

    else:
        empty_li_month += [i for i in range(1,13)]
        empty_li_year += [year] * 12

monthly_market_data = pd.DataFrame(columns=["year","quarter","month","index_monthly_return","monthly_trade_value","monthly_usd_return","market_pe","diff_nima_usd"])
monthly_market_data["year"] = empty_li_year
monthly_market_data["month"] = empty_li_month

for year in range(first_year, last_year + 1):

    if year == first_year:
        for month in range(first_years_month, 13):
            idx = monthly_market_data[(monthly_market_data.year == year) & (monthly_market_data.month == month)].index.item()
            index_ret = return_calculator(market_df[(market_df["month"] == month) & (market_df["year"] == year)]["overall_index"])
            usd_ret = return_calculator(market_df[(market_df["month"] == month) & (market_df["year"] == year)]["usd"])
            monthly_trade_value = market_df[(market_df["month"] == month) & (market_df["year"] == year)]["trade_value"].mean()
            month_pe_ttm = market_df[(market_df["month"] == month) & (market_df["year"] == year)]["market_pe"].dropna().mean()
            diff_nima_usd_monthly = market_df[(market_df["month"] == month) & (market_df["year"] == year)]["diff_nima_usd"].dropna().median()
            monthly_market_data["quarter"][idx] = market_df[(market_df["month"] == month) & (market_df["year"] == year)].quarter.mean()
            monthly_market_data["index_monthly_return"][idx] = index_ret
            monthly_market_data["monthly_usd_return"][idx] = usd_ret
            monthly_market_data["monthly_trade_value"][idx] = monthly_trade_value
            monthly_market_data["market_pe"][idx] = month_pe_ttm
            monthly_market_data["diff_nima_usd"][idx] = diff_nima_usd_monthly

    elif year == last_year:
        for month in range(1, last_years_month + 1):
            idx = monthly_market_data[(monthly_market_data.year == year) & (monthly_market_data.month == month)].index.item()
            index_ret = return_calculator(market_df[(market_df["month"] == month) & (market_df["year"] == year)]["overall_index"])
            usd_ret = return_calculator(market_df[(market_df["month"] == month) & (market_df["year"] == year)]["usd"])
            monthly_trade_value = market_df[(market_df["month"] == month) & (market_df["year"] == year)]["trade_value"].mean()
            month_pe_ttm = market_df[(market_df["month"] == month) & (market_df["year"] == year)]["market_pe"].dropna().mean()
            diff_nima_usd_monthly = market_df[(market_df["month"] == month) & (market_df["year"] == year)]["diff_nima_usd"].dropna().median()
            monthly_market_data["quarter"][idx] = market_df[(market_df["month"] == month) & (market_df["year"] == year)].quarter.mean()
            monthly_market_data["index_monthly_return"][idx] = index_ret
            monthly_market_data["monthly_usd_return"][idx] = usd_ret
            monthly_market_data["monthly_trade_value"][idx] = monthly_trade_value
            monthly_market_data["market_pe"][idx] = month_pe_ttm
            monthly_market_data["diff_nima_usd"][idx] = diff_nima_usd_monthly

    else:
        for month in range(1, 13):
            idx = monthly_market_data[(monthly_market_data.year == year) & (monthly_market_data.month == month)].index.item()
            index_ret = return_calculator(market_df[(market_df["month"] == month) & (market_df["year"] == year)]["overall_index"])
            usd_ret = return_calculator(market_df[(market_df["month"] == month) & (market_df["year"] == year)]["usd"])
            monthly_trade_value = market_df[(market_df["month"] == month) & (market_df["year"] == year)]["trade_value"].mean()
            month_pe_ttm = market_df[(market_df["month"] == month) & (market_df["year"] == year)]["market_pe"].dropna().mean()
            diff_nima_usd_monthly = market_df[(market_df["month"] == month) & (market_df["year"] == year)]["diff_nima_usd"].dropna().median()
            monthly_market_data["quarter"][idx] = market_df[(market_df["month"] == month) & (market_df["year"] == year)].quarter.mean()
            monthly_market_data["index_monthly_return"][idx] = index_ret
            monthly_market_data["monthly_usd_return"][idx] = usd_ret
            monthly_market_data["monthly_trade_value"][idx] = monthly_trade_value
            monthly_market_data["market_pe"][idx] = month_pe_ttm
            monthly_market_data["diff_nima_usd"][idx] = diff_nima_usd_monthly

data = pd.read_excel(market_other_data_path + "//" + 'inflation.xlsx')
data["inflation"] = data["CPI"].pct_change()
data = data.drop(["CPI"],axis=1)
data = data.dropna()
data["date"] = data["date"].jalali.parse_jalali("%Y-%m")
data["month"] = data["date"].jalali.month
data["year"] = data["date"].jalali.year
data["quarter"] = data["date"].jalali.quarter
data["weekday"] = data["date"].jalali.weekday
data["day"] = data["date"].jalali.day
inflation_df = data

data = pd.read_excel(market_other_data_path + "//" + 'money_supply.xlsx')
data["change_in_money_supply"] = data["money_supply"].pct_change()
data = data.drop(["money_supply"],axis=1)
data = data.dropna()
data["date"] = data["date"].jalali.parse_jalali("%Y-%m")
data["month"] = data["date"].jalali.month
data["year"] = data["date"].jalali.year
data["quarter"] = data["date"].jalali.quarter
data["weekday"] = data["date"].jalali.weekday
data["day"] = data["date"].jalali.day
money_supply_df = data

monthly_market_data = monthly_market_data.merge(inflation_df,on=["year","month"],how="inner")
monthly_market_data = monthly_market_data.merge(money_supply_df,on=["year","month"],how="inner")
monthly_market_data = monthly_market_data.drop(["quarter_x","date_x","quarter_y","weekday_x","day_x","date_y","weekday_y","day_y","quarter"],axis=1)
monthly_market_data.rename(columns = {'monthly_trade_value':"market_monthly_trade_value"}, inplace = True)

monthly_market_data.to_csv(market_data_processed_path + "\\" + "market_data_processed.csv")


In [8]:
dataset_df = pd.DataFrame()
for file_name in list_of_sector_processed_files:
    data = pd.read_excel(sectors_data_processed_path+ "\\" + file_name)
    data = data.fillna(method='ffill')
    data = data.drop(["Unnamed: 0"],axis=1)
    market_data = pd.read_csv(market_data_processed_path+ "\\" + list_of_market_processed_files[0])
    market_data = market_data.drop(["Unnamed: 0"],axis=1)
    market_data = market_data.fillna(method='ffill')
    new_data = data.merge(market_data,on=["year","month"],how="inner")

    

    if len(dataset_df) == 0:
        dataset_df = new_data

    else:
        dataset_df = pd.concat([dataset_df,new_data])

dataset_df = dataset_df.sort_values(by = ['year', 'quarter'], ascending = [True, True])
dataset_df.to_csv(dataset_path + "\\" + "dataset.csv")

In [13]:
return_calculator(new_data["monthly_return"].iloc[0:5])

0.12320328542094394

In [61]:
number_of_rows = len(new_data)
new_data["sector"] = [file_name.split(".")[0]] * number_of_rows
new_data["next_6m_return"] = [np.nan] * number_of_rows
new_data["next_3m_return"] = [np.nan] * number_of_rows
new_data["next_1m_return"] = [np.nan] * number_of_rows

new_data["last_6m_return"] = [np.nan] * number_of_rows
new_data["last_3m_return"] = [np.nan] * number_of_rows
new_data["last_1m_return"] = [np.nan] * number_of_rows

new_data["next_6m_usd_return"] = [np.nan] * number_of_rows
new_data["next_3m_usd_return"] = [np.nan] * number_of_rows
new_data["next_1m_usd_return"] = [np.nan] * number_of_rows

new_data["last_6m_usd_return"] = [np.nan] * number_of_rows
new_data["last_3m_usd_return"] = [np.nan] * number_of_rows
new_data["last_1m_usd_return"] = [np.nan] * number_of_rows

new_data["next_6m_index_return"] = [np.nan] * number_of_rows
new_data["next_3m_index_return"] = [np.nan] * number_of_rows
new_data["next_1m_index_return"] = [np.nan] * number_of_rows

new_data["last_6m_index_return"] = [np.nan] * number_of_rows
new_data["last_3m_index_return"] = [np.nan] * number_of_rows
new_data["last_1m_index_return"] = [np.nan] * number_of_rows

new_data["relative_trade_value"] = [np.nan] * number_of_rows
new_data["market_relative_trade_value"] = [np.nan] * number_of_rows

for index, row in new_data.iterrows():
    if (index <=5) or (index > number_of_rows - 6):
        continue
    else:
        new_data["next_6m_return"].iloc[index] = return_calculator(new_data["monthly_return"].iloc[index:index+6])
        new_data["next_3m_return"].iloc[index] = return_calculator(new_data["monthly_return"].iloc[index:index+3])
        new_data["next_1m_return"].iloc[index] = return_calculator(new_data["monthly_return"].iloc[index:index+1])

        new_data["last_6m_return"].iloc[index] = return_calculator(new_data["monthly_return"].iloc[index-6:index])
        new_data["last_3m_return"].iloc[index] = return_calculator(new_data["monthly_return"].iloc[index-3:index])
        new_data["last_1m_return"].iloc[index] = return_calculator(new_data["monthly_return"].iloc[index-1:index])

        new_data["next_6m_usd_return"].iloc[index] = return_calculator(new_data["monthly_usd_return"].iloc[index:index+6])
        new_data["next_3m_usd_return"].iloc[index] = return_calculator(new_data["monthly_usd_return"].iloc[index:index+3])
        new_data["next_1m_usd_return"].iloc[index] = return_calculator(new_data["monthly_usd_return"].iloc[index:index+1])

        new_data["last_6m_usd_return"].iloc[index] = return_calculator(new_data["monthly_usd_return"].iloc[index-6:index])
        new_data["last_3m_usd_return"].iloc[index] = return_calculator(new_data["monthly_usd_return"].iloc[index-3:index])
        new_data["last_1m_usd_return"].iloc[index] = return_calculator(new_data["monthly_usd_return"].iloc[index-1:index])

        new_data["next_6m_index_return"].iloc[index] = return_calculator(new_data["index_monthly_return"].iloc[index:index+6])
        new_data["next_3m_index_return"].iloc[index] = return_calculator(new_data["index_monthly_return"].iloc[index:index+3])
        new_data["next_1m_index_return"].iloc[index] = return_calculator(new_data["index_monthly_return"].iloc[index:index+1])

        new_data["last_6m_index_return"].iloc[index] = return_calculator(new_data["index_monthly_return"].iloc[index-6:index])
        new_data["last_3m_index_return"].iloc[index] = return_calculator(new_data["index_monthly_return"].iloc[index-3:index])
        new_data["last_1m_index_return"].iloc[index] = return_calculator(new_data["index_monthly_return"].iloc[index-1:index])

        new_data["relative_trade_value"].iloc[index] = new_data["monthly_trade_value"].iloc[index] / new_data["monthly_trade_value"].iloc[index-6:index].mean() - 1
        new_data["market_relative_trade_value"].iloc[index] = new_data["market_monthly_trade_value"].iloc[index] / new_data["market_monthly_trade_value"].iloc[index-6:index].mean() - 1

to_winsorize_list = [
       'P/E-ttm','market_pe', 'diff_nima_usd', 'inflation',
       'change_in_money_supply', 'next_6m_return', 'last_6m_return',
       'next_3m_return', 'next_1m_return', 'last_3m_return', 'last_1m_return',
       'next_6m_usd_return', 'next_3m_usd_return', 'next_1m_usd_return',
       'last_6m_usd_return', 'last_3m_usd_return', 'last_1m_usd_return',
       'next_6m_index_return', 'next_3m_index_return', 'next_1m_index_return',
       'last_6m_index_return', 'last_3m_index_return', 'last_1m_index_return',
       'excess_next_1m_return', 'excess_next_3m_return','excess_next_6m_return',
        'relative_trade_value','market_relative_trade_value']

for column in to_winsorize_list:
    new_data[column] = winsorize(new_data[column],(0.025,0.025))    

new_data["excess_next_1m_return"] = new_data["next_1m_return"] - new_data["next_1m_index_return"]
new_data["excess_next_3m_return"] = new_data["next_3m_return"] - new_data["next_3m_index_return"]
new_data["excess_next_6m_return"] = new_data["next_6m_return"] - new_data["next_6m_index_return"]

new_data = new_data.dropna()

In [63]:
new_data.head(20)

,year,quarter,month,monthly_return,monthly_trade_value,P/E-ttm,index_monthly_return,market_monthly_trade_value,monthly_usd_return,market_pe,...,next_1m_index_return,last_6m_index_return,last_3m_index_return,last_1m_index_return,sector,excess_next_1m_return,excess_next_3m_return,excess_next_6m_return,relative_trade_value,market_relative_trade_value
12,1383,1,2,-0.110998,5.292591e+09,20.821571,-0.023139,5.733787e+11,0.006186,9.204991,...,-0.023139,0.346579,0.080239,0.037037,کانی های فلزی,-0.087859,-0.205511,-0.311130,-0.192719,0.597023
13,1383,1,3,-0.085911,1.428462e+09,18.008974,0.057844,3.424835e+11,0.003644,9.091843,...,0.057844,0.217809,0.059658,-0.023139,کانی های فلزی,-0.143755,-0.185656,-0.265709,-0.782132,-0.203167
14,1383,2,4,0.167920,1.446658e+10,17.932564,0.117313,3.920871e+11,0.015168,9.830817,...,0.117313,0.180425,0.071640,0.057844,کانی های فلزی,0.050607,0.076226,-0.141899,1.743635,-0.081367
15,1383,2,5,-0.057940,3.280709e+09,19.551565,0.007987,3.348734e+11,0.000601,10.512494,...,0.007987,0.247238,0.154594,0.117313,کانی های فلزی,-0.065927,-0.104273,-0.158353,-0.433366,-0.253579
16,1383,2,6,0.140091,6.934991e+09,24.158027,0.046103,2.331924e+11,0.002558,10.606916,...,0.046103,0.262459,0.191383,0.007987,کانی های فلزی,0.093988,-0.068613,-0.075501,0.215301,-0.362130
17,1383,3,7,-0.135864,8.280634e+09,25.011535,-0.020934,2.584528e+11,0.000786,10.587055,...,-0.020934,0.262563,0.178160,0.046103,کانی های فلزی,-0.114930,-0.174476,-0.167756,0.302076,-0.244238
18,1383,3,8,0.005780,2.034530e+09,23.648377,0.034463,4.380707e+11,0.001389,10.377778,...,0.034463,0.191985,0.032385,-0.020934,کانی های فلزی,-0.028683,-0.057463,-0.003758,-0.692390,0.231419
19,1383,3,9,-0.039080,1.897181e+09,23.667963,-0.003128,6.609523e+11,0.002160,10.823565,...,-0.003128,0.262273,0.059502,0.034463,کانی های فلزی,-0.035953,-0.011377,0.092416,-0.687500,0.983690
20,1383,4,10,-0.017344,3.074538e+08,22.980166,-0.023326,4.559971e+11,0.008036,10.109833,...,-0.023326,0.189518,0.009641,-0.003128,کانی های فلزی,0.005981,-0.011845,0.335873,-0.910227,0.180509
21,1383,4,11,-0.020694,3.582420e+08,22.966363,-0.038547,1.759394e+11,0.000107,9.980988,...,-0.038547,0.039791,0.007174,-0.023326,کانی های فلزی,0.017854,0.055163,0.417547,-0.905458,-0.556742
